In [1]:
!pip install datasets transformers torch tqdm rouge-score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-1

In [2]:
from datasets import load_dataset

naits_lfqa = load_dataset('fahdsoliman/naits_lfqa_processed')
old_lfqa = load_dataset('fahdsoliman/lfqa_with_supports_subset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/286 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/108 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_name = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [4]:
from datasets import concatenate_datasets

train_data = concatenate_datasets([naits_lfqa['train'], naits_lfqa['validation']])

In [ ]:
train_data[0]

{'answer': 'If you do not pay the extension fee for your shared hosting, it will be paused for three months. During this period, you will not have access to the hosting, and will not be able to retrieve any data. If the fees due are not paid within these three months, the hosting will be permanently cancelled, and you will not have the right to request any backup copy of the hosting data. According to Article (41), the procedure begins with temporarily stopping the service for a period of up to three months, and if the fees are not paid, the hosting is permanently canceled without the possibility of recovering data. If the hosting is affiliated with a public sector entity, the procedures are slightly different, as the authority sends a financial claim a month before the due date to remind the entity of the necessity of renewal.',
 'question': 'i have shared hosting that i reserved a year ago, but i received an email saying that this hosting has been suspended due to non-payment of the 

In [5]:
from datasets import concatenate_datasets

preprocessed_dataset = concatenate_datasets([naits_lfqa['test'], old_lfqa['test']])
preprocessed_dataset2 = old_lfqa['test']

In [6]:
# evaluate function

from datasets import load_metric, load_dataset
from tqdm.auto import tqdm
import torch


def evaluate_model(preprocessed_dataset):
    metric_rouge = load_metric("rouge", trust_remote_code=True)
    metric_bleu = load_metric("sacrebleu", trust_remote_code=True)
    all_predictions = []
    all_labels = []
    model.eval()

    for example in tqdm(preprocessed_dataset):
        # إعداد المدخلات
        inputs = tokenizer("question: {} context: {}".format(example["question"], example["context"]), truncation=True, padding=True, return_tensors="pt")
        # print(inputs)
        reference = example["answer"]

        # توليد التنبؤات
        with torch.no_grad():
            outputs = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),
                                    min_length=64,
                                    max_length=256,
                                    do_sample=False,
                                    early_stopping=True,
                                    num_beams=8,
                                    temperature=1.0,
                                    top_k=None,
                                    top_p=None,
                                    eos_token_id=tokenizer.eos_token_id,
                                    no_repeat_ngram_size=3,
                                    num_return_sequences=1)

        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # تجميع التنبؤات والمراجع
        all_predictions.append(prediction)
        all_labels.append(reference)

    # حساب ROUGE
    rouge_result = metric_rouge.compute(predictions=all_predictions, references=all_labels, use_stemmer=True)
    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}

    # حساب BLEU
    bleu_result = metric_bleu.compute(predictions=all_predictions, references=[[ref] for ref in all_labels])
    bleu_score = bleu_result["score"]

    print("ROUGE Scores:", rouge_scores)
    print("BLEU Score:", bleu_score)

In [7]:
import numpy as np
label_pad_token_id = -100
def tokenize_dataset(examples):
    inputs = ["question: {} context: {}".format(q, c) for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, padding='max_length', truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=360, padding='max_length', truncation=True,
                            return_tensors="np")

    model_inputs["decoder_input_ids"] = labels["input_ids"][:, :-1].tolist()
    # replace pad_token_id with label_pad_token_id to avoid loss calculation on those tokens
    labels["input_ids"] = np.where(labels["input_ids"] == tokenizer.pad_token_id,
                                    label_pad_token_id, labels["input_ids"])

    model_inputs["labels"] = labels["input_ids"][:, 1:].tolist()
    return model_inputs

# tokenized_datasets = {}
# for split in ["train", "validation"]:
#         tokenized_datasets[split] = combined_data[split].map(tokenize_dataset,
#                                                     batched=True,
#                                                     remove_columns=combined_data[split].column_names,
#                                                     desc="Preparing dataset for tokenization")

In [8]:
tokenized_datasets = {}

tokenized_datasets = train_data.map(tokenize_dataset,
                                            batched=True,
                                            remove_columns=train_data.column_names,
                                            desc="Preparing dataset for tokenization")

Preparing dataset for tokenization:   0%|          | 0/85 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
train_dataset = tokenized_datasets
train_dataset.set_format(type='torch')


In [ ]:
print(tokenized_datasets[0])

{'input_ids': tensor([    0, 40018,    35,  ...,     1,     1,     1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'decoder_input_ids': tensor([    0,  1106,    47,   109,    45,   582,     5,  5064,  4029,    13,
          110,  1373,  5162,     6,    24,    40,    28, 28115,    13,   130,
          377,     4,  1590,    42,   675,     6,    47,    40,    45,    33,
          899,     7,     5,  5162,     6,     8,    40,    45,    28,   441,
            7, 22661,   143,   414,     4,   318,     5,  3154,   528,    32,
           45,  1199,   624,   209,   130,   377,     6,     5,  5162,    40,
           28, 13140,  8102,     6,     8,    47,    40,    45,    33,     5,
          235,     7,  2069,   143,  8010,  5375,     9,     5,  5162,   414,
            4,   767,     7,  6776,    36,  4006,   238,     5,  7089,  3772,
           19,  8059,  8197,     5,   544,    13,    10,   675,     9,    62,
            7,   130,   377,     6,     8,   114,     5,  3154,    32,    4

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # Few-shot learning typically requires fewer epochs
    per_device_train_batch_size=4,
    learning_rate=1e-6,  # Adjust as needed
    weight_decay=0.00,
    evaluation_strategy="no",  # No evaluation during few-shot learning
    save_strategy="no",  # No saving during few-shot learning
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=22, training_loss=2.238347140225497, metrics={'train_runtime': 49.0502, 'train_samples_per_second': 1.733, 'train_steps_per_second': 0.449, 'total_flos': 184203891179520.0, 'train_loss': 2.238347140225497, 'epoch': 1.0})

In [12]:
evaluate_model(preprocessed_dataset2)
evaluate_model(preprocessed_dataset)


<ipython-input-6-593f2c4d81ba>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_rouge = load_metric("rouge", trust_remote_code=True)


  0%|          | 0/108 [00:00<?, ?it/s]

ROUGE Scores: {'rouge1': 27.42508657992797, 'rouge2': 6.063596368624494, 'rougeL': 14.82509975036423, 'rougeLsum': 14.79777042885264}
BLEU Score: 0.9419014101971015


  0%|          | 0/125 [00:00<?, ?it/s]

ROUGE Scores: {'rouge1': 29.210024749432844, 'rouge2': 7.532604039573558, 'rougeL': 16.358400478834493, 'rougeLsum': 16.373522536881183}
BLEU Score: 1.6261724957783734


In [13]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
# it all starts with a question/query
query = "If I do not pay the service fees within a month and the service is temporarily suspended, can I obtain the information I have on the hosting service?"

# given the question above suppose these documents below were found in some document store
documents = ["Article (45) fees: Hosting service fees are collected according to the fees approved by the Authority. Article (46) fees: The Authority has the right to amend the fees for the hosting service, and the new fees for the service are applied upon the first renewal or request for modification of the current hosting by the customer. Article (47) fees The Authority adopts a pre-payment system for all fees related to the hosting service, Article (48) fees: Public sector entities are excluded from the provisions of Article 44 in this document, as the Authority sends a financial claim after booking the hosting to these entities, provided that the payment notification is received within One month from the date of reservation, Article (49) Fees: The customer can pay location hosting fees every six months, provided that this is mentioned in the location hosting contract.",
             ]



# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)


["If you don't pay the service fees within a month and the service is temporarily suspended, can you obtain the information you have on the hosting service? Yes, you can get the information if you pay the fees within the period of suspension. If you do not pay the fee within a period of time, then the service will be temporarily suspended and the information will be lost."]

In [14]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
query = "Why does water heated to room temperature feel colder than the air around it?"

# given the question above suppose these documents below were found in some document store
documents = ["when the skin is completely wet. The body continuously loses water by...",
             "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling'...",
             "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway...",
             "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example ...",
             "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid ..."]

# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)



['The air around the water is colder than the air around it. This is due to the difference in temperature between the water and the air, and the difference between the air and the water. If the air is warmer than the water, then the water will feel colder, because the air will have a higher temperature.']